# Inferencia con YOLO y Evaluación de Métricas
En este notebook, realizaremos inferencia usando un modelo YOLO preentrenado para detectar objetos en imágenes. También calcularemos métricas como IoU, precisión, recall y mAP para evaluar la calidad de las detecciones.

## Instalación de la Librería YOLO
Utilizaremos la librería `ultralytics`, que simplifica el uso de YOLOv5/v8. Esta librería permite realizar entrenamiento, inferencia y evaluación de manera eficiente.

In [ ]:
!pip install ultralytics

## Importar Librerías y Configurar el Modelo
Cargaremos un modelo YOLO preentrenado y lo utilizaremos para realizar inferencias.

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np
import requests

# Cargar un modelo YOLO preentrenado
model = YOLO('yolov8n.pt')  # Modelo pequeño preentrenado en COCO

## Cargar y Procesar Imágenes
Seleccionaremos imágenes de ejemplo para realizar la detección de objetos.

In [ ]:
url1 = 'https://images.pexels.com/photos/210182/pexels-photo-210182.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1'
url2 = 'https://images.pexels.com/photos/30192151/pexels-photo-30192151/free-photo-of-traditional-dance-in-sarajevo-celebration.jpeg'

def download_image(url):
    response = requests.get(url, stream=True).raw
    image = cv2.imdecode(np.asarray(bytearray(response.read()), dtype=np.uint8), cv2.IMREAD_COLOR)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image1 = download_image(url1)
image2 = download_image(url2)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].imshow(image1)
axes[0].axis('off')
axes[0].set_title('Imagen 1')
axes[1].imshow(image2)
axes[1].axis('off')
axes[1].set_title('Imagen 2')
plt.show()

## Inferencia con YOLO
Realizaremos la detección de objetos en las imágenes cargadas y visualizaremos los resultados con cajas delimitadoras y etiquetas.

In [ ]:
# Realizar detección en tus imágenes
results1 = model(image1)
results2 = model(image2)

# Obtener las cajas predichas (predictions)
pred_boxes1 = [box.xyxy[0].tolist() for box in results1[0].boxes] if results1[0].boxes is not None else []
pred_boxes2 = [box.xyxy[0].tolist() for box in results2[0].boxes] if results2[0].boxes is not None else []

# Visualizar resultados para la primera imagen
annotated_image1 = results1[0].plot()
plt.imshow(annotated_image1)
plt.axis('off')
plt.title('Detecciones - Imagen 1')
plt.show()

# Visualizar resultados para la segunda imagen
annotated_image2 = results2[0].plot()
plt.imshow(annotated_image2)
plt.axis('off')
plt.title('Detecciones - Imagen 2')
plt.show()

## Evaluación de Métricas
Calcularemos métricas clave para evaluar la calidad de las detecciones realizadas por YOLO:

In [ ]:
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea) if float(boxAArea + boxBArea - interArea) > 0 else 0

def calculate_precision_recall(pred_boxes, true_boxes, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    
    for pred_box in pred_boxes:
        matched = False
        for true_box in true_boxes:
            iou = calculate_iou(pred_box, true_box)
            if iou >= iou_threshold:
                tp += 1
                matched = True
                break
        if not matched:
            fp += 1
    fn = len(true_boxes) - tp
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

def calculate_average_precision(precisions, recalls):
    precisions = np.array([0] + precisions + [0])
    recalls = np.array([0] + recalls + [1])
    for i in range(len(precisions) - 1, 0, -1):
        precisions[i - 1] = max(precisions[i - 1], precisions[i])
    indices = np.where(recalls[1:] != recalls[:-1])[0]
    return np.sum((recalls[indices + 1] - recalls[indices]) * precisions[indices + 1])

### Cargando caja real de la imagen2


La anotación está creada con una herramienta que genera los recuadros con formato COCO, entonces hay que transformas este recuadro para que el modelo lo cargue correctamente.

In [ ]:
def transform_bbox(bbox):
    """
    Transforma una bounding box del formato [x_min, y_min, width, height]
    al formato [x_min, y_min, x_max, y_max].
    """
    x_min, y_min, width, height = bbox
    x_max = x_min + width
    y_max = y_min + height
    return [x_min, y_min, x_max, y_max]

bboxes_coco = [
    [881.9361084220718, 469.2352371732813, 1353.9980638915815, 2450.7647628267187],
    [2298.121974830587, 350.5130687318486, 1218.3155856728022, 2569.4869312681512],
    [16.96030977734753, 678.4123910939009, 1048.7124878993227, 2241.587608906099],
    [3550.3581800580805, 734.9467570183938, 829.6418199419195, 2185.053242981606]
]

# Convertir todas las bounding boxes
true_boxes2 = [transform_bbox(bbox) for bbox in bboxes_coco]
print("Bounding Boxes transformadas:", true_boxes2)

Visualizamos la imagen con las cajas reales

In [ ]:
def draw_boxes(image, boxes, title="Imagen con Bounding Boxes"):
    # Crear una copia de la imagen para no modificar la original
    img_with_boxes = image.copy()
    
    # Dibujar cada caja delimitadora
    for box in boxes:
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(img_with_boxes, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 20)  # Color azul
        
    # Mostrar la imagen con las cajas
    plt.imshow(img_with_boxes)
    plt.axis("off")
    plt.title(title)
    plt.show()


# Visualizar las cajas para Imagen 2
draw_boxes(image2, true_boxes2, title="Ground Truth - Imagen 2")

<strong>NOTA</strong>: En este notebook, nos enfocamos en evaluar únicamente las predicciones correspondientes a la clase "person" (ID 0 en COCO). Esto evita penalizar al modelo por predicciones correctas de otras clases y asegura que las métricas reflejen el rendimiento para la clase objetivo.

A menudo, los modelos como YOLO detectan múltiples clases de objetos en una imagen. Sin embargo, si nuestro interés está en evaluar el rendimiento del modelo para una sola clase específica (por ejemplo, "person"), debemos filtrar las predicciones para excluir otras clases.
La función filter_predictions_by_class permite hacer esto fácilmente. Se proporciona el ID de la clase objetivo (en este caso, 0 para "person") y devuelve únicamente las cajas correspondientes a esa clase.

In [ ]:
def filter_predictions_by_class(results, class_id):
    """
    Filtra las predicciones por clase específica.
    Args:
    - results: Predicciones del modelo YOLO (formato ultralytics).
    - class_id: ID de la clase que se desea filtrar (ej. 0 para "person").
    Returns:
    - Lista de cajas filtradas en formato [xmin, ymin, xmax, ymax].
    """
    filtered_boxes = []
    for box in results.boxes:
        if int(box.cls) == class_id:  # Compara con el ID de clase deseado
            filtered_boxes.append(box.xyxy[0].tolist())
    return filtered_boxes

# Clase "person" tiene ID 0 en el conjunto COCO
CLASS_ID_PERSON = 0

# Filtrar predicciones de la clase "person" para ambas imágenes
filtered_pred_boxes2 = filter_predictions_by_class(results2[0], CLASS_ID_PERSON)


### Ejemplo de Evaluación
Vamos a calcular precisión, recall y mAP para la segunda imagen usando cajas reales.

In [ ]:
# Cálculo de métricas
precision2, recall2 = calculate_precision_recall(filtered_pred_boxes2, true_boxes2)
average_precision = calculate_average_precision([precision2], [recall2])
print(f'Precision: {precision2:.2f}, Recall: {recall2:.2f}, mAP: {average_precision:.2f}')

In [ ]:
def draw_boxes_side_by_side(image, true_boxes, pred_boxes, true_color=(0, 255, 0), pred_color=(255, 0, 0), thickness=3):
    """
    Dibuja las bounding boxes reales y predichas en imágenes separadas y las muestra juntas.
    Args:
    - image: Imagen original.
    - true_boxes: Lista de cajas reales (ground truth) en formato [xmin, ymin, xmax, ymax].
    - pred_boxes: Lista de cajas predichas en formato [xmin, ymin, xmax, ymax].
    - true_color: Color de las cajas reales (por defecto verde).
    - pred_color: Color de las cajas predichas (por defecto rojo).
    - thickness: Grosor de los recuadros.
    """
    # Crear copias de la imagen original
    img_with_true_boxes = image.copy()
    img_with_pred_boxes = image.copy()
    
    # Dibujar las cajas reales (ground truth)
    for box in true_boxes:
        xmin, ymin, xmax, ymax = map(int, box)
        cv2.rectangle(img_with_true_boxes, (xmin, ymin), (xmax, ymax), true_color, thickness)
    
    # Dibujar las cajas predichas
    for box in pred_boxes:
        xmin, ymin, xmax, ymax = map(int, box)
        cv2.rectangle(img_with_pred_boxes, (xmin, ymin), (xmax, ymax), pred_color, thickness)
    
    # Mostrar las imágenes lado a lado
    _ , axes = plt.subplots(1, 2, figsize=(15, 7))
    axes[0].imshow(img_with_true_boxes)
    axes[0].axis('off')
    axes[0].set_title("Ground Truth - Imagen 2")
    
    axes[1].imshow(img_with_pred_boxes)
    axes[1].axis('off')
    axes[1].set_title("Predicciones - Imagen 2")
    
    plt.show()

# Mostrar la imagen 2 con recuadros reales y predichos lado a lado
draw_boxes_side_by_side(
    image2,
    true_boxes2,          # Cajas reales
    filtered_pred_boxes2,      # Cajas predichas filtradas
    true_color=(0, 255, 0),  # Verde para ground truth
    pred_color=(255, 0, 0)   # Rojo para predicciones
)
